### Jupyter that shows how to perform symbolic regression of a given function

In [1]:
weights_path = "path to weight"

In [ ]:
## Load architecture, set into eval mode, and pass the config parameters
  model = Model.load_from_checkpoint(model_path, cfg=cfg.architecture)